In [1]:
# imports
import yaml
import pandas as pd
import os.path
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
#import apply_donor_constraints
from sklearn.cluster import KMeans, DBSCAN, Birch
from sklearn_extra.cluster import KMedoids
import math
import hdbscan

import my_utils

In [2]:
# read configuration (algorithm parameters etc) into dictionary
with open('../data/config.yaml', 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# read donor attributes
dtAttrDonor = pd.read_csv('../data/all_attrs_donors.csv')
dtAttrDonor['tag'] = 'donor'

# read receiver attributes
dtAttrReceiver = pd.read_csv('../data/all_attrs_receivers.csv')
dtAttrReceiver['tag'] = 'receiver'

# if donors overlap with receivers (i.e., they share the same hydrofabric files), exclude them from the receivers
if config['hydrofabric']['shared']:
    dtAttrReceiver = dtAttrReceiver.loc[~dtAttrReceiver['id'].isin(dtAttrDonor['id'])]

# combine donor & receiver attributes
dtAttrAll = pd.concat([dtAttrDonor, dtAttrReceiver])

# detemine whether the catchment is snowy (as snowy and non-snowy catchments are processed separately)
dtAttrAll['snowy'] = dtAttrAll['snow_frac'].apply(lambda x: True if x >= config['pars']['general']['minSnowFrac'] else False)
del dtAttrDonor, dtAttrReceiver

# columns in the attrs table that are not actual attributes
config['non_attr_cols'] = ["id","tag","snowy","hsg"]

# pretty print nested dictionaries
# print(yaml.dump(config, default_flow_style=False)) 

# compute spatial distance between all receivers and donors if not already exists
f1 = '../data/dist_spatial_donor_receiver.csv'
if os.path.isfile(f1):
    dist_spatial = pd.read_csv(f1,index_col=0)
else:
    pass
    #distSpatial0 = compute_dist_spatial(f1, subset(dtAttrAll,tag=="donor")$id, subset(dtAttrAll,tag=="receiver")$id)

In [4]:
scenario = 'camels'
dtDonorAll = pd.DataFrame()    

# attributes to be used
attrs1 = config['attrs'][scenario]

# all receivers to find donors for
recs0 = dtAttrAll[dtAttrAll['tag']=='receiver']['id'].tolist()
recs1 = list() #recerivers that have already been processed in previous rounds

# reduce the attribute table to attributes
dtAttr0 = dtAttrAll[config['non_attr_cols']+attrs1]

# figure out valid attributes to use this round
dtAttr = my_utils.get_valid_attrs(recs0, recs1, dtAttr0, attrs1, config)

# apply principal component analysis
myscores, weights = my_utils.apply_pca(dtAttr.drop(config['non_attr_cols'], axis=1), config['pars']['general']['minVarPCA'])    
print("weights: " + str(weights))
        
# donors and receivers for this round
receiversAll1 = dtAttr[dtAttr['tag']=='receiver']['id'].tolist()

# determine which receivers to be processed for the current round
# process only those not-yet processed receivers
recs = [r1 for r1 in recs0 if r1 in receiversAll1 and r1 not in recs1]
recs1 = recs1 + recs
print(str(len(recs)) + " receivers to be processed this round")  


Excluding 4 attributes: sand_frac,clay_frac,soil_porosity,soil_conductivity
Number of PCs selected: 6
PCA total portion of variance explained ... 0.839470613290475
weights: [0.39203068 0.27393743 0.12728765 0.08450586 0.06471906 0.0575193 ]
168 receivers to be processed this round


In [5]:
# process snowy and non-snowy catchments sparately
snow1 = False
method = 'hdbscan'
    
# the current snowy group
dtAttr1 = dtAttr[dtAttr['snowy']==snow1]
donors = dtAttr1[dtAttr1['tag']=='donor']['id'].tolist()
receivers = dtAttr1[dtAttr1['tag']=='receiver']['id'].tolist()
receivers = [x for x in receivers if x in recs]

# scores for donors and and receivers for this round
# keep1 = (dtAttr1.tag=="donor") | ((dtAttr1.tag=="receiver") & (dtAttr1.id.isin(receivers)))  
# dtAttr1 = dtAttr1[keep1.values]     
# scores1 = myscores[(dtAttr['snowy']==snow1).values][keep1.values]

scores1 = myscores[(dtAttr['snowy']==snow1).values]

In [ ]:
# identify donors iteratively
dfDonors = pd.DataFrame()
kround = 0
donors1 = donors.copy()
nrec_with_donor0 = dfDonors.shape[0]
labels = np.zeros(scores1.shape[0])
label0 = -99 #labels indicating donor identified

while dfDonors.shape[0] < len(receivers):
    
    kround = kround + 1
    print('\n=========== Round=' + str(kround) + ' =====================')
    
    label_rec, count_rec = np.unique(labels[len(donors):], return_counts=True) # receiver clusters
    label_don, count_don = np.unique(labels[:len(donors)], return_counts=True) # donor clusters
    count_rec = count_rec[label_rec != label0]; label_rec = label_rec[label_rec != label0]
    
    print('label_rec: ' + str(label_rec))
    
    labels1 = labels.copy()
    for ll in label_rec:
        donors1 = [x for jj,x in enumerate(donors) if labels1[jj]==ll]
        receivers1 =[ x for jj,x in enumerate(receivers) if labels1[jj+len(donors)]==ll]
        
        print('Number of donors in cluster ' + str(ll) + ': ' + str(len(donors1)))
        
        if len(donors1)>0:
            if len(donors1) > config['pars'][method]['nDonorMax']:
                idx1 = [donors.index(x) for x in donors1]
                idx2 = [receivers.index(x) for x in receivers1]
                idx0 = idx1 + [x+len(donors) for x in idx2]
                mydata1 = scores1.iloc[idx0]
                
                fit1 = hdbscan.HDBSCAN(min_samples=11,min_cluster_size=config['pars'][method]['minClusterSize'],allow_single_cluster=False).fit(mydata1)
                fit1.labels_ = fit1.labels_ + 2
                label_rec1, count_rec1 = np.unique(fit1.labels_[len(donors1):], return_counts=True) # receiver clusters
                label_don1, count_don1 = np.unique(fit1.labels_[:len(donors1)], return_counts=True) # donor clusters
                labels[idx0] = fit1.labels_ + labels1.max()
                print("donor clusters: " + str(label_don1))
                print("donor cluster counts: " + str(count_don1))
                print("receiver clusters: " + str(label_rec1))
                print("receiver cluster counts: " + str(count_rec1))
            else:
                dfDonors = pd.concat((dfDonors, my_utils.assign_donors(scenario, donors1, receivers1, config['pars']['general'], dist_spatial, dtAttrAll)),axis=0)
                labels[labels == ll] = label0  
        else: # note proximity chooses from all donors
            dfDonors = pd.concat((dfDonors, my_utils.assign_donors('proximity', donors, receivers1, config['pars']['general'], dist_spatial, dtAttrAll)),axis=0)            
            labels[labels == ll] = label0           
       
    # Algorithm converged or no donors available for next round
    if (nrec_with_donor0 == dfDonors.shape[0]) & (kround>3):        
        recs2 = [x for x in receivers if x not in dfDonors['id']]
        if len(recs2) > 0:
            print("\nAlgorithm converged with donors unidentified for some receivers ... use proximity for these receivers")
            dfDonors = pd.concat((dfDonors, my_utils.assign_donors('proximity', donors, recs2, config['pars']['general'], dist_spatial, dtAttrAll)),axis=0)    
            
    # update progress
    print("Number of receivers with donors identified: " + str(dfDonors.shape[0]))
    if dfDonors.shape[0] > 0:
        uniq, freq = np.unique(dfDonors['tag'],return_counts=True)
        print(dict(zip(uniq, freq)))

    # update number of receivers with donors identified
    nrec_with_donor0 = dfDonors.shape[0]
    

           

In [11]:
recs2 = [x for x in receivers if x not in dfDonors['id'].tolist()]
print(len(recs2))
print(len(receivers))
print(len(dfDonors['id']))

4
68
64
